<img height=200 width=200 align='left' src='https://avatars2.githubusercontent.com/u/365630?v=3&s=400'>
<img height=200 width=200 align='right' src='https://upload.wikimedia.org/wikipedia/en/7/74/TensorFlow.png'>
<img height=50 width=50 align='middle' src='http://bowriversolutions.com/wp-content/uploads/2014/04/VS-Icon-315x400.png'>

<img src='http://orig13.deviantart.net/a0c9/f/2009/339/f/4/shadic_vs_nazo_by_oskarmandude.gif'>

In this notebook we will run a logistic regression from sklearn and compare its perfomance on the iris data set 
with that of a tensorflow model

In [37]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score
from __future__ import division
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf

<h2 style="color:#2980b9"> Preparing the data </h2>

In [85]:
data = datasets.load_iris()
random_indices = np.random.choice(range(len(data.data)),replace=False,size=len(data.data))

In [96]:
X = data.data[random_indices]
X = scale(X)
y = data.target[random_indices]

In [97]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [98]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, train_size=.8, test_size=.2)

<h2 style="color:#16a085"> Le Regression Logistique by SKLEARN </h2>

In [99]:
clf = LogisticRegression(fit_intercept=False)

In [100]:
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

In [101]:
print "Accuracy score: {0:.2f}".format(accuracy_score(y_test, preds))

Accuracy score: 0.90


<h2 style="color:#9b5ab6"> Das Logistic Regression by TENSORFLOW </h2>

In [102]:
enc = OneHotEncoder()
enc.fit([[0],[1],[2]])

OneHotEncoder(categorical_features='all', dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [110]:
y_train_hot = enc.transform(np.vstack(y_train)).toarray().astype(np.float32)
y_test_hot = enc.transform(np.vstack(y_test)).toarray().astype(np.float32)

In [111]:
print y_train[0:5]
print '===> \n'
print y_train_hot[0:5]
print '\n'
print y_test[0:5]
print '===>\n'
print y_test_hot[0:5]

[ 0.  1.  1.  2.  2.]
===> 

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


[ 0.  2.  2.  0.  1.]
===>

[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]]


### Initiating the Graph

In [124]:
num_labels = np.shape(y_train_hot)[1]

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.constant(X_train)
    tf_train_labels = tf.constant(y_train_hot)
    tf_valid_dataset = tf.constant(X_test)
  
    # Variables
    weights = tf.Variable(
      tf.truncated_normal([np.shape(X_train)[1], num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    beta = 0.00
  
    # Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*tf.nn.l2_loss(weights)
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
  
    # Predictions for the training, validation, and test data
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)

### Training the Graph

In [138]:
num_steps = 10000

def accuracy(predictions, labels):
    return (np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

saver = tf.train.Saver([weights],keep_checkpoint_every_n_hours=2)

with tf.Session(graph=graph) as session: 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        _, l = session.run([optimizer, loss])
        if (step % 2000 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Validation accuracy: %.2f%%' % accuracy(valid_prediction.eval(), y_test_hot))
            preds_tf = valid_prediction.eval()
    saver.save(session, 'vs_sklearn', global_step = step)
    
    

Initialized
Loss at step 0: 1.570057
Validation accuracy: 0.23%
Loss at step 2000: 0.541924
Validation accuracy: 0.93%
Loss at step 4000: 0.425406
Validation accuracy: 0.93%
Loss at step 6000: 0.378345
Validation accuracy: 0.97%
Loss at step 8000: 0.349923
Validation accuracy: 0.97%


<h2 align=middle> And the winner is ... </h2>

<p align='middle'><img src='https://upload.wikimedia.org/wikipedia/en/7/74/TensorFlow.png'></p>

<img src='http://orig06.deviantart.net/bfe7/f/2012/172/f/7/kawaii_plum_by_nyanqueen-d54az71.gif'>